In [1]:
import tvm
from tvm.script import ir as I
from tvm.script import tir as T
import os
from tvm import autotvm, auto_scheduler
from tvm.autotvm.tuner import XGBTuner, GATuner, RandomTuner, GridSearchTuner
from tvm import meta_schedule as ms
from tvm.ir import IRModule
from tvm import relax
vocab_size = 49984
@I.ir_module
class ModuleSrc:
    @T.prim_func
    def main(lv1323: T.Buffer((T.int64(512), T.int64(vocab_size)), "uint32"), lv1324: T.Buffer((T.int64(128), T.int64(vocab_size)), "float16"), lv1607: T.Buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16"), p_output0_intermediate: T.Buffer((T.int64(1), T.int64(1), T.int64(vocab_size)), "float32")):
        T.func_attr({"global_symbol": "main", "tir.noalias": T.bool(True)})
        # with T.block("root"):
        p_output0_intermediate_1 = T.alloc_buffer((T.int64(4096), T.int64(vocab_size)), "float16")
        var_matmul_intermediate = T.alloc_buffer((T.int64(1), T.int64(1), T.int64(vocab_size)), "float16")
        for i, j in T.grid(T.int64(4096), T.int64(vocab_size)):
            with T.block("decode"):
                v_i, v_j = T.axis.remap("SS", [i, j])
                T.reads(lv1323[v_i // T.int64(8), v_j], lv1324[v_i // T.int64(32), v_j])
                T.writes(p_output0_intermediate_1[v_i, v_j])
                p_output0_intermediate_1[v_i, v_j] = (T.Cast("float16", T.bitwise_and(T.shift_right(lv1323[v_i // T.int64(8), v_j], T.Cast("uint32", v_i % T.int64(8)) * T.uint32(4)), T.uint32(15))) - T.float16(7)) * lv1324[v_i // T.int64(32), v_j]
        for i0, i1, i2, k in T.grid(T.int64(1), T.int64(1), T.int64(vocab_size), T.int64(4096)):
            with T.block("matmul"):
                v_i0, v_i1, v_i2, v_k = T.axis.remap("SSSR", [i0, i1, i2, k])
                T.reads(lv1607[v_i0, v_i1, v_k], p_output0_intermediate_1[v_k, v_i2])
                T.writes(var_matmul_intermediate[v_i0, v_i1, v_i2])
                with T.init():
                    var_matmul_intermediate[v_i0, v_i1, v_i2] = T.float16(0)
                var_matmul_intermediate[v_i0, v_i1, v_i2] = var_matmul_intermediate[v_i0, v_i1, v_i2] + lv1607[v_i0, v_i1, v_k] * p_output0_intermediate_1[v_k, v_i2]
        for i0, i1, i2 in T.grid(T.int64(1), T.int64(1), T.int64(vocab_size)):
            with T.block("compute"):
                v_i0, v_i1, v_i2 = T.axis.remap("SSS", [i0, i1, i2])
                T.reads(var_matmul_intermediate[v_i0, v_i1, v_i2])
                T.writes(p_output0_intermediate[v_i0, v_i1, v_i2])
                p_output0_intermediate[v_i0, v_i1, v_i2] = T.Cast("float32", var_matmul_intermediate[v_i0, v_i1, v_i2])


In [2]:
import numpy as np
target = tvm.target.Target("opencl -device=adreno", host="llvm -mtriple=aarch64-linux-gnu")
device_key="android"
rpc_host = "10.158.176.30"
rpc_port = 5001
# remote = autotvm.measure.request_remote(device_key, "10.158.176.30", 5001, timeout=10000)
# dev = remote.device(str(target), 0)

# num_flop = 1228406784
# W_np = np.random.uniform(size=(512, vocab_size)).astype("uint32")
# S_np = np.random.uniform(size=(128, vocab_size)).astype("float16")
# Input_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
# # Output_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
# W_nd = tvm.nd.array(W_np, dev)
# S_nd = tvm.nd.array(S_np, dev)
# Input_nd = tvm.nd.array(Input_np, dev)
# Output_nd = tvm.nd.array(np.zeros((1, 1, vocab_size), dtype="float32"), dev)

In [3]:
rpc_config = ms.runner.RPCConfig(tracker_host=rpc_host, tracker_port=rpc_port, tracker_key = device_key)
runner= ms.runner.RPCRunner(rpc_config)
# ms.builder.LocalBuilder()
sch = tvm.tir.Schedule(ModuleSrc)
database = ms.tune_tir(
    mod=ModuleSrc,
    target=target,
    max_trials_global=64,
    num_trials_per_iter=64,
    work_dir="./tune_first",
    cost_model="xgb",
    runner = runner
)
print(len(database))
sch1 = ms.tir_integration.compile_tir(database, sch.mod, target)
print(type(sch1))

====-==-=-=-=-==
====-==-=-=-=-==
====-==-=-=-=-==
2023-08-02 09:07:53 [INFO] Logging directory: ./tune_first/logs


TVMError: Traceback (most recent call last):
  7: 0xffffffffffffffff
  6: 0x00000000005b7412
  5: __libc_start_main
  4: ffi_call
  3: tvm::meta_schedule::TuneContextNode::Initialize()
        at /data/workspace/llm/github/new_wksp/tvm-unity/src/meta_schedule/tune_context.cc:58
  2: tvm::meta_schedule::PostOrderApplyNode::InitializeWithTuneContext(tvm::meta_schedule::TuneContext const&)
        at /data/workspace/llm/github/new_wksp/tvm-unity/src/meta_schedule/space_generator/post_order_apply.cc:44
  1: tvm::meta_schedule::SpaceGeneratorNode::InitializeWithTuneContext(tvm::meta_schedule::TuneContext const&)
        at /data/workspace/llm/github/new_wksp/tvm-unity/src/meta_schedule/space_generator/space_generator.cc:91
  0: tvm::meta_schedule::GetRuleKindFromTarget(tvm::Target const&)
        at /data/workspace/llm/github/new_wksp/tvm-unity/src/meta_schedule/space_generator/space_generator.cc:82
  File "/data/workspace/llm/github/new_wksp/tvm-unity/src/meta_schedule/space_generator/spac

In [4]:
from tvm.script import relax as R
@I.ir_module
class Module:
    @R.function
    def main(A: R.Tensor((3, 4), dtype="float32"), B: R.Tensor((4, 5), dtype="float32")):
        with R.dataflow():
            lv: R.Tensor((3, 5), dtype="float32") = R.matmul(A, B)
            gv: R.Tensor((3, 5), dtype="float32") = lv
            R.output(gv)
        return gv

In [5]:
## auto_scheduler test
from tvm import auto_scheduler
import numpy as np
a_np = np.random.rand(3, 4).astype("float32")
b_np = np.random.rand(4, 5).astype("float32")
a_nd = tvm.runtime.NDArray(a_np)
b_nd = tvm.runtime.NDArray(b_np)
sch = tvm.tir.Schedule(Module)

params = {"A": a_np, "B": b_np}
## 报错，这里只支持relay
# tasks = auto_scheduler.extract_tasks(sch.mod, params, target=target)
tasks = ms.relax_integration.extract_tasks(sch.mod, target=target, params=params)
print(len(tasks))

0


In [6]:

from mod_deploy import Module as ModuleAll
params_all = {}
tasks_all = auto_scheduler.extract_tasks(ModuleAll, params_all, target=target)
print(len(tasks_all))

TVMError: Task extraction had the following errors:
Traceback (most recent call last):
  File "/data/workspace/llm/github/new_wksp/tvm-unity/python/tvm/auto_scheduler/relay_integration.py", line 71, in call_all_topi_funcs
    compiler.lower(mod, target)
  File "/data/workspace/llm/github/new_wksp/tvm-unity/python/tvm/relay/backend/vm.py", line 126, in lower
    self._lower(mod, raw_targets)
  File "/data/workspace/llm/github/new_wksp/tvm-unity/python/tvm/_ffi/_ctypes/packed_func.py", line 238, in __call__
    raise get_last_ffi_error()
tvm.error.InternalError: Traceback (most recent call last):
  12: 0xffffffffffffffff
  11: clone
  10: start_thread
        at /build/glibc-S7Ft5T/glibc-2.23/nptl/pthread_create.c:333
  9: ffi_call
  8: operator()
        at /data/workspace/llm/github/new_wksp/tvm-unity/src/relay/backend/vm/compiler.cc:834
  7: tvm::relay::vm::VMCompiler::Lower(tvm::IRModule, tvm::runtime::Array<tvm::Target, void> const&)
        at /data/workspace/llm/github/new_wksp/tv

In [7]:
import numpy as np
log_file = "tune.json"
def _detect_local_cuda():
    dev = tvm.cuda()
    if not dev.exist:
        return None
    return tvm.target.Target(
        {
            "kind": "cuda",
            "max_shared_memory_per_block": dev.max_shared_memory_per_block,
            "max_threads_per_block": dev.max_threads_per_block,
            "thread_warp_size": dev.warp_size,
            "registers_per_block": 65536,
            "arch": "sm_" + tvm.cuda().compute_version.replace(".", ""),
        }
    )
# target = tvm.target.Target("cuda", host="llvm")
target = _detect_local_cuda()

print(target)
# 定义计算任务
dev = tvm.cuda(0)

num_flop = 1228406784
W_np = np.random.uniform(size=(512, vocab_size)).astype("uint32")
S_np = np.random.uniform(size=(128, vocab_size)).astype("float16")
Input_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
# Output_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
W_nd = tvm.nd.array(W_np, dev)
S_nd = tvm.nd.array(S_np, dev)
Input_nd = tvm.nd.array(Input_np, dev)
Output_nd = tvm.nd.array(np.zeros((1, 1, vocab_size), dtype="float32"), dev)
sch = tvm.tir.Schedule(ModuleSrc)
new_mod = sch.mod


cuda -keys=cuda,gpu -arch=sm_61 -max_num_threads=1024 -max_shared_memory_per_block=49152 -max_threads_per_block=1024 -registers_per_block=65536 -thread_warp_size=32


In [20]:
# task = auto_scheduler.SearchTask(func=sch.mod['fused_fused_decode11_fused_matmul5_cast2'], args=sch.mod['fused_fused_decode11_fused_matmul5_cast2'].params, target=target)

# tune_option = auto_scheduler.TuningOptions(
#     num_measure_trials=10,
#     measure_callbacks=[auto_scheduler.RecordToFile(log_file)],
#     verbose=2,
# )


database = ms.tune_tir(
    mod=new_mod,
    target=target,
    max_trials_global=64,
    num_trials_per_iter=64,
    work_dir="./tune_45593_1",
    cost_model="xgb"
)
print(len(database))
sch1 = ms.tir_integration.compile_tir(database, new_mod, target)
print(type(sch1))

====-==-=-=-=-==
====-==-=-=-=-==
====-==-=-=-=-==
2023-08-01 15:46:14 [INFO] Logging directory: ./tune_45593_1/logs
2023-08-01 15:46:14 [INFO] LocalBuilder: max_workers = 4
2023-08-01 15:46:14 [INFO] LocalRunner: max_workers = 1
2023-08-01 15:46:17 [INFO] [task_scheduler.cc:159] Initializing Task #0: "main"


,Name,FLOP,Weight,Speed (GFLOPS),Latency (us),Weighted Latency (us),Trials,Done
0,main,1228406784,1,N/A,N/A,N/A,0,



Total trials: 0
Total latency (us): 0

2023-08-01 15:46:17 [DEBUG] [task_scheduler.cc:318] 
 ID | Name |       FLOP | Weight | Speed (GFLOPS) | Latency (us) | Weighted Latency (us) | Trials | Done 
---------------------------------------------------------------------------------------------------------
  0 | main | 1228406784 |      1 |            N/A |          N/A |                   N/A |      0 |      
---------------------------------------------------------------------------------------------------------
Total trials: 0
Total latency (us): 0

2023-08-01 15:46:17 [INFO] [task_scheduler.cc:180] TaskScheduler picks Task #0: "main"
2023-08-01 15:46:37 [INFO] [task_scheduler.cc:193] Sending 64 sample(s) to builder
2023-08-01 15:46:48 [INFO] [task_scheduler.cc:195] Sending 64 sample(s) to runner
2023-08-01 15:47:31 [DEBUG] XGB iter   0: tr-p-rmse: 0.384314	tr-a-peak@32: 0.959011	tr-rmse: 0.550808	tr-rmse: 0.550808
2023-08-01 15:47:31 [DEBUG] XGB iter  25: tr-p-rmse: 0.081095	tr-a-peak

,Name,FLOP,Weight,Speed (GFLOPS),Latency (us),Weighted Latency (us),Trials,Done
0,main,1228406784,1,63.7432,19271.1677,19271.1677,64,


2023-08-01 15:47:32 [DEBUG] [task_scheduler.cc:318] 
 ID | Name |       FLOP | Weight | Speed (GFLOPS) | Latency (us) | Weighted Latency (us) | Trials | Done 
---------------------------------------------------------------------------------------------------------
  0 | main | 1228406784 |      1 |        63.7432 |   19271.1677 |            19271.1677 |     64 |      
---------------------------------------------------------------------------------------------------------
Total trials: 64
Total latency (us): 19271.2


Total trials: 64
Total latency (us): 19271.2

2023-08-01 15:47:32 [INFO] [task_scheduler.cc:260] Task #0 has finished. Remaining task(s): 0


,Name,FLOP,Weight,Speed (GFLOPS),Latency (us),Weighted Latency (us),Trials,Done
0,main,1228406784,1,63.7432,19271.1677,19271.1677,64,Y


2023-08-01 15:47:32 [DEBUG] [task_scheduler.cc:318] 
 ID | Name |       FLOP | Weight | Speed (GFLOPS) | Latency (us) | Weighted Latency (us) | Trials | Done 
---------------------------------------------------------------------------------------------------------
  0 | main | 1228406784 |      1 |        63.7432 |   19271.1677 |            19271.1677 |     64 |    Y 
---------------------------------------------------------------------------------------------------------
Total trials: 64
Total latency (us): 19271.2


Total trials: 64
Total latency (us): 19271.2

1598
<class 'tvm.tir.schedule.schedule.Schedule'>


In [21]:
# print(sch1.trace)
# print(sch1.mod.script())
rt_mod = tvm.build(sch1.mod, target="cuda")

evaluator = rt_mod.time_evaluator("main", dev, number=100)

print("evaluator GEMV-Blocking: %f GFLOPS" % (1228406784 / evaluator(W_nd, S_nd, Input_nd, Output_nd).mean / 1e9))




evaluator GEMV-Blocking: 62.644059 GFLOPS


In [23]:

record_database = ms.Database.create(kind='json', work_dir='./tune_45593_1')


In [24]:
record_sch = ms.tir_integration.compile_tir(record_database, new_mod, target)

record_rt_mod = tvm.build(record_sch.mod, target="cuda")

record_evaluator = record_rt_mod.time_evaluator("main", dev, number=20)

print("evaluator GEMV-Blocking: %f GFLOPS" % (num_flop / record_evaluator(W_nd, S_nd, Input_nd, Output_nd).mean / 1e9))
print(record_sch.trace)
print(record_sch.mod.script())

evaluator GEMV-Blocking: 56.786827 GFLOPS
# from tvm import tir
def apply_trace(sch: tir.Schedule) -> None:
  b0 = sch.get_block(name="decode", func_name="main")
  b1 = sch.get_block(name="matmul", func_name="main")
  b2 = sch.get_block(name="compute", func_name="main")
  b3 = sch.get_block(name="root", func_name="main")
  sch.annotate(block_or_loop=b1, ann_key="meta_schedule.tiling_structure", ann_val="SSSRRSRS")
  l4, l5, l6, l7 = sch.get_loops(block=b1)
  v8, v9, v10, v11, v12 = sch.sample_perfect_tile(loop=l4, n=5, max_innermost_factor=64, decision=[1, 1, 1, 1, 1])
  l13, l14, l15, l16, l17 = sch.split(loop=l4, factors=[v8, v9, v10, v11, v12], preserve_unit_iters=True)
  v18, v19, v20, v21, v22 = sch.sample_perfect_tile(loop=l5, n=5, max_innermost_factor=64, decision=[1, 1, 1, 1, 1])
  l23, l24, l25, l26, l27 = sch.split(loop=l5, factors=[v18, v19, v20, v21, v22], preserve_unit_iters=True)
  v28, v29, v30, v31, v32 = sch.sample_perfect_tile(loop=l6, n=5, max_innermost_factor=64, de

## lm_head(32000, 4096) 原始优化性能

In [12]:
@I.ir_module
class ModuleOpted:
    @T.prim_func
    def main(lv1323: T.Buffer((512, 32000), "uint32"), lv1324: T.Buffer((128, 32000), "float16"), lv1607: T.Buffer((1, 1, 4096), "float16"), p_output0_intermediate: T.Buffer((1, 1, 32000), "float32")):
        T.func_attr({"global_symbol": "main", "tir.noalias": T.bool(True)})
        # with T.block("root"):
        var_matmul_intermediate_local = T.alloc_buffer((1, 1, 32000), "float16", scope="local")
        var_matmul_intermediate_rf_local = T.alloc_buffer((16, 1, 1, 32000), "float16", scope="local")
        for ax0_fused_0 in T.thread_binding(2000, thread="blockIdx.x"):
            for ax0_fused_1 in T.thread_binding(16, thread="threadIdx.x"):
                for ax1_0_fused_1 in T.thread_binding(16, thread="threadIdx.y"):
                    with T.block("matmul_rf_init"):
                        vax1_0_fused_1 = T.axis.spatial(16, ax1_0_fused_1)
                        v0 = T.axis.spatial(32000, ax0_fused_0 * 16 + ax0_fused_1)
                        T.reads()
                        T.writes(var_matmul_intermediate_rf_local[vax1_0_fused_1, 0, 0, v0])
                        var_matmul_intermediate_rf_local[vax1_0_fused_1, 0, 0, v0] = T.float16(0)
                    for ax1_0_fused_0, ax1_1 in T.grid(32, 8):
                        with T.block("matmul_rf_update"):
                            vax1_0_fused_1 = T.axis.spatial(16, ax1_0_fused_1)
                            v0 = T.axis.spatial(32000, ax0_fused_0 * 16 + ax0_fused_1)
                            vax1_0_fused_0, vax1_1 = T.axis.remap("RR", [ax1_0_fused_0, ax1_1])
                            T.reads(var_matmul_intermediate_rf_local[vax1_0_fused_1, 0, 0, v0], lv1607[0, 0, vax1_0_fused_0 * 128 + vax1_0_fused_1 * 8 + vax1_1], lv1323[(vax1_0_fused_0 * 128 + vax1_0_fused_1 * 8 + vax1_1) // 8, v0], lv1324[(vax1_0_fused_0 * 128 + vax1_0_fused_1 * 8 + vax1_1) // 32, v0])
                            T.writes(var_matmul_intermediate_rf_local[vax1_0_fused_1, 0, 0, v0])
                            var_matmul_intermediate_rf_local[vax1_0_fused_1, 0, 0, v0] = var_matmul_intermediate_rf_local[vax1_0_fused_1, 0, 0, v0] + lv1607[0, 0, vax1_0_fused_0 * 128 + vax1_0_fused_1 * 8 + vax1_1] * ((T.Cast("float16", T.bitwise_and(T.shift_right(lv1323[(vax1_0_fused_0 * 128 + vax1_0_fused_1 * 8 + vax1_1) // 8, v0], T.Cast("uint32", (vax1_0_fused_0 * 128 + vax1_0_fused_1 * 8 + vax1_1) % 8) * T.uint32(4)), T.uint32(15))) - T.float16(7)) * lv1324[(vax1_0_fused_0 * 128 + vax1_0_fused_1 * 8 + vax1_1) // 32, v0])
            for ax1_fused in T.thread_binding(16, thread="threadIdx.x"):
                for ax0 in T.thread_binding(16, thread="threadIdx.y"):
                    with T.block("matmul"):
                        vax1_0_fused_1 = T.axis.reduce(16, ax0)
                        v0 = T.axis.spatial(32000, ax0_fused_0 * 16 + ax1_fused)
                        T.reads(var_matmul_intermediate_rf_local[vax1_0_fused_1, 0, 0, v0])
                        T.writes(var_matmul_intermediate_local[0, 0, v0])
                        with T.init():
                            var_matmul_intermediate_local[0, 0, v0] = T.float16(0)
                        var_matmul_intermediate_local[0, 0, v0] = var_matmul_intermediate_local[0, 0, v0] + var_matmul_intermediate_rf_local[vax1_0_fused_1, 0, 0, v0]
            for ax0_fused in T.thread_binding(16, thread="threadIdx.x"):
                with T.block("compute"):
                    v0 = T.axis.spatial(32000, ax0_fused_0 * 16 + ax0_fused)
                    T.reads(var_matmul_intermediate_local[0, 0, v0])
                    T.writes(p_output0_intermediate[0, 0, v0])
                    p_output0_intermediate[0, 0, v0] = T.Cast("float32", var_matmul_intermediate_local[0, 0, v0])

sch_opted = tvm.tir.Schedule(ModuleOpted)
rt_mod_opted = tvm.build(sch_opted.mod, target="cuda")
evaluator_opted = rt_mod_opted.time_evaluator("main", dev, number=20)

print("evaluator_opted GEMV-Blocking: %f GFLOPS" % (num_flop / evaluator_opted(W_nd, S_nd, Input_nd, Output_nd).mean / 1e9))


TVMError: Traceback (most recent call last):
  5: 0xffffffffffffffff
  4: 0x00000000005b7412
  3: __libc_start_main
  2: ffi_call
  1: operator()
        at /data/workspace/llm/github/new_wksp/tvm-unity/src/runtime/profiling.cc:879
  0: operator()
        at /data/workspace/llm/github/new_wksp/tvm-unity/src/runtime/library_module.cc:90
  File "/data/workspace/llm/github/new_wksp/tvm-unity/src/runtime/library_module.cc", line 87
TVMError: Assert fail: T.Cast("int32", main_lv1323_handle_shape[1]) == 32000, Argument main.lv1323_handle.shape[1] has an unsatisfied constraint: 32000 == T.Cast("int32", main_lv1323_handle_shape[1])

In [22]:
## 未优化版本
# vocab_size = vocab_size
num_flop = 1228406784
W_np = np.random.uniform(size=(512, vocab_size)).astype("uint32")
S_np = np.random.uniform(size=(128, vocab_size)).astype("float16")
Input_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
# Output_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
W_nd = tvm.nd.array(W_np, dev)
S_nd = tvm.nd.array(S_np, dev)
Input_nd = tvm.nd.array(Input_np, dev)
Output_nd = tvm.nd.array(np.zeros((1, 1, vocab_size), dtype="float32"), dev)
@I.ir_module
class ModuleUnOpt:
    @T.prim_func
    def main(lv1323: T.Buffer((T.int64(512), T.int64(vocab_size)), "uint32"), lv1324: T.Buffer((T.int64(128), T.int64(vocab_size)), "float16"), lv1607: T.Buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16"), p_output0_intermediate: T.Buffer((T.int64(1), T.int64(1), T.int64(vocab_size)), "float32")):
        T.func_attr({"global_symbol": "main", "tir.noalias": True})
        # with T.block("root"):
        p_output0_intermediate_1 = T.alloc_buffer((T.int64(4096), T.int64(vocab_size)), "float16")
        var_matmul_intermediate = T.alloc_buffer((T.int64(1), T.int64(1), T.int64(vocab_size)), "float16")
        for i, j in T.grid(T.int64(4096), T.int64(vocab_size)):
            with T.block("decode"):
                v_i, v_j = T.axis.remap("SS", [i, j])
                T.reads(lv1323[v_i // T.int64(8), v_j], lv1324[v_i // T.int64(32), v_j])
                T.writes(p_output0_intermediate_1[v_i, v_j])
                p_output0_intermediate_1[v_i, v_j] = (T.Cast("float16", T.bitwise_and(T.shift_right(lv1323[v_i // T.int64(8), v_j], T.Cast("uint32", v_i % T.int64(8)) * T.uint32(4)), T.uint32(15))) - T.float16(7)) * lv1324[v_i // T.int64(32), v_j]
        for i0, i1, i2, k in T.grid(T.int64(1), T.int64(1), T.int64(vocab_size), T.int64(4096)):
            with T.block("matmul"):
                v_i0, v_i1, v_i2, v_k = T.axis.remap("SSSR", [i0, i1, i2, k])
                T.reads(lv1607[v_i0, v_i1, v_k], p_output0_intermediate_1[v_k, v_i2])
                T.writes(var_matmul_intermediate[v_i0, v_i1, v_i2])
                with T.init():
                    var_matmul_intermediate[v_i0, v_i1, v_i2] = T.float16(0)
                var_matmul_intermediate[v_i0, v_i1, v_i2] = var_matmul_intermediate[v_i0, v_i1, v_i2] + lv1607[v_i0, v_i1, v_k] * p_output0_intermediate_1[v_k, v_i2]
        for i0, i1, i2 in T.grid(T.int64(1), T.int64(1), T.int64(vocab_size)):
            with T.block("compute"):
                v_i0, v_i1, v_i2 = T.axis.remap("SSS", [i0, i1, i2])
                T.reads(var_matmul_intermediate[v_i0, v_i1, v_i2])
                T.writes(p_output0_intermediate[v_i0, v_i1, v_i2])
                p_output0_intermediate[v_i0, v_i1, v_i2] = T.Cast("float32", var_matmul_intermediate[v_i0, v_i1, v_i2])
    
sch_unopt = tvm.tir.Schedule(ModuleUnOpt)
# sch_unopt_mod = relax.transform.RewriteCUDAGraph()(sch_unopt.mod)
print(target)
### 将原始代码转为gpu代码
with target:
    sch_unopt_mod = tvm.tir.transform.DefaultGPUSchedule()(sch_unopt.mod)
# print(sch_unopt_mod.script())
rt_mod_unopt = tvm.build(sch_unopt_mod, target="cuda")
evaluator_unopt = rt_mod_unopt.time_evaluator("main", dev, number=100)

print("evaluator_opted GEMV-Blocking: %f GFLOPS" % (num_flop / evaluator_unopt(W_nd, S_nd, Input_nd, Output_nd).mean / 1e9))

cuda -keys=cuda,gpu -arch=sm_61 -max_num_threads=1024 -max_shared_memory_per_block=49152 -max_threads_per_block=1024 -registers_per_block=65536 -thread_warp_size=32
evaluator_opted GEMV-Blocking: 32.723813 GFLOPS
